# EE 380L: Data Mining
## Assignment 3

Due: Feb 28 at 11:59 pm  
Total points: 55

You may work in pairs. Only one student per pair needs to submit the assignment on Canvas, but you should include names and UTEIDs for both students.

## Liheng Ding(ld24529), Yu Sun(ys8797)

# Question 1: Ridge and Lasso Regression using R (15 points)
To perform Ridge and Lasso regression in R, we will use the [glmnet](https://cran.r-project.org/web/packages/glmnet/vignettes/glmnet_beta.html#lin) package.

In this question, we use the diabetes dataset again. The dataset can be found at: http://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt

After importing the datasets, use the following code to add additional interaction variables to the features matrix. You should have 65 variables (including 55 interaction variables) and one target variable.

In this question, you will explore the application of Lasso and Ridge regression using  package in R. The following code will split the data into training and testing sets.

(a) (3 pts) Use the ["pairs"](http://stat.ethz.ch/R-manual/R-devel/library/graphics/html/pairs.html) function to see the scatter-plots of pairs of (the original set of 10) variables.

(b) (5 pts) Use glmnet ridge and lasso regression models to do a 5-fold cross validation using [cvTools](https://cran.r-project.org/web/packages/cvTools/index.html) package. For the sweep of the regularization parameter, we will look at a grid of values ranging from $\lambda = 10^{10}$ to $\lambda = 10^{-2}$. In R, you can consider this range of values as follows:

      alphas <- 10^seq(from = -2, to = 10, length.out = 100)

  Report the best chosen $\lambda$ based on cross validation. The cross validation should happen on your training data using  average MSE as the scoring metric.

(c) (4 pts) Run ridge and lasso for all of the alphas specified above (on training data), and plot the coefficients learned for each of them - there should be one plot each for lasso and ridge, so a total of two plots; the plots for different features for a method should be on the same plot (e.g. Fig 6.6 of JW). What do you qualitatively observe when value of the regularization parameter is changed?

(d) (3 pts) Run least squares regression, ridge, and lasso on the training data. For ridge and lasso, use only the best regularization parameter. Report the prediction error (MSE) on the test data for each.

# Question 2: Multi-level Model in Python (18 points)
In this problem, you will explore multi-level model using a python package [PyMC3](https://pymc-devs.github.io/pymc3/index.html#). The dataset "oxboys.csv" will be used in this question.  This dataset contains three fields:
- Individual ID
- Age in years
- Height in cm

(a)  (2 pts) Plot the relationship between height and year, and draw a linearly regressed line ignoring the ID variable.

(b)  (2 pts) Plot the relationship between height and year, but this time, fit a different linear regression for each individual.

(c)  (2 + 4 + 6 pts) Divide the dataset into training and test sets.  The training set contains the first 7 years of the measurements, and the test set contains the rest of the measurements. 
Build three different linear models:
- Global model:  a linear model ignoring the id variable. Pool all data and estimate one common regression model to assess the influence of Age across all Individuals' heights.
- Local model:  a different linear model for each individual i.e., 26 different linear regressions. We are interested in whether different individuals actually follow separate regression models.
- Multilevel model:  Use the [PyMC3](http://pymc-devs.github.io/pymc3/notebooks/GLM-hierarchical.html#Partial-pooling:-Hierarchical-Regression-aka,-the-best-of-both-worlds) package to fit a multilevel model specified as follows:

\begin{align*}
\text{height}_{it} &= \beta_{0i} + \beta_{1i} \text{year}_{it} + \epsilon_{it}\\
\beta_{0i} &= \beta_{00}  + \eta_{0i} \\
\beta_{1i} &= \beta_{10} + \eta_{1i} \\
\begin{bmatrix} \eta_{0i} \\ \eta_{1i} \end{bmatrix} &\sim \text{Bivariate Normal}(\begin{bmatrix} 0 \\ 0 \end{bmatrix}, \begin{bmatrix} \sigma_{1}^2 & 0\\ 0 & \sigma_{2}^2 \end{bmatrix})\\
\epsilon_{it} &\sim \text{Normal}(0, \sigma^2)
\end{align*}

Predict the heights for the next 2 years, and calculate the mean squared errors from the three models.

(d)  (2 pts) Briefly state what do $\beta_{00}$ and $\beta_{10}$ mean in this multilevel model.

# Question 3: Sparse Linear Regression (15 points)
In this question, we will explore a couple of modelling techniques for sparse linear regression. For this question,
please make sure you have $numpy/scipy$ and $sklearn$ installed. Each student is to expected to run 3 sparse regression models:
- Lasso (from the sklearn package)
- Automatic Relevance Determination (ARD), available in the sklearn package, or use the provided file $iterative\_ard.py$.
- SubmodRegression. See provided file $submodRegression.py$. The function run submodregression
takes as input the training data, test data and a matrix $C$ which is to be set as identity except for the fMRI dataset for which it is provided as detailed below.

(5 + 5 + 5 pts) You are only expected to run the above 3 models on one of the following 4 datasets. Take the numeral part of your
UTEID, and divide it by 4. Use the remainder to select the corresponding dataset you need to work with:-

0. Simulated data: filename is $simulated.npz$. Use $data = numpy.load(simulated.npz)$ to access the serialized file. Then access features using $data[’X’]$, and target variable using $data[’y’]$. Report $R^2$ for $k = 50, 100, 150$ on 5-fold cross validation. Use $C =identity$ for submodRegression. What happens if you change $C$ to a different diagonal matrix?

1. fMRI data. Use $numpy.load$ like above to load the variables. Remember to use $data[’C’].item()$ to access the sparse csc matrix. Report $R^2 $for sparsity $k = {50, 100, 200}$.

2. KDD cup 2008 dataset from https://archive.ics.uci.edu/ml/datasets/KDD+Cup+1998+Data.
Use $cup98lrn.zip$ for both training/test. Randomly subsample 10, 000 rows to use as the training data, and another 5,000 rows to use as the test data. Replace all missing values with 0. Report $R^2$ for sparsity $k = {50, 100, 200}$. Use $C=identity$ for submodRegression.

3. Blog feedback data.See https://archive.ics.uci.edu/ml/datasets/BlogFeedback for details.
Randomly subsample 10,000 rows to use as the training data, and another 5,000 rows to use as the test data. Report $R^2$ for sparsity $k = {50, 100, 200}$.

No.1 dataset

1) Lasso

In [1]:
%matplotlib inline
# from __future__ import print_function
# print(__doc__)

import numpy as np
import matplotlib.pyplot as plt
import sys

from sklearn import datasets, linear_model
from sklearn import datasets
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lasso
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

data=np.load("Q3/simulated.npz")
X = data['X']
y = data['y']

from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state=20)

k_fold = KFold(5)
alphas =  10**np.linspace(10,-2,1000)*0.5
lasso = Lasso()
print ("Lasso:")
for alpha in alphas:
    lasso.set_params(alpha=alpha, random_state=20)
    lasso.fit(X_train, y_train)
    k = 0
    for coef in lasso.coef_:
        if coef != 0:
            k = k + 1
    if k == 50 or k == 100 or k== 150:
        r_square = 0
        for i, (train, test) in enumerate(k_fold.split(X_train, y_train)):
            lasso.fit(X_train[train], y_train[train])
            r_square = r_square + lasso.score(X_train[test], y_train[test])
        print ("k=%d, R-square=%f"% (k, r_square/5))   

/Library/Python/2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Lasso:
k=50, R-square=0.621039
k=100, R-square=0.965618
k=100, R-square=0.967176
k=100, R-square=0.968654
k=150, R-square=0.995615


2) ARD

In [2]:
from sklearn.linear_model import ARDRegression
print ("ARDRegression")

def ard_FindK(X_train, y_train, k, thresholds):
    ard = ARDRegression(compute_score=True)
    for threshold in thresholds:
        ard.set_params(threshold_lambda=threshold)
        ard.fit(X_train, y_train)
        k_count = 0
        for coef in ard.coef_:
            if coef != 0:
                k_count = k_count + 1
        if abs(k_count - k)<=5:
            r_square = 0
            for i, (train, test) in enumerate(k_fold.split(X_train, y_train)):
                ard.fit(X_train[train], y_train[train])
                r_square = r_square + ard.score(X_train[test], y_train[test])
            print ("k=%d, R-square=%f"% (k, r_square/5))           

ard_FindK(X_train, y_train, 50, 10 ** np.linspace(0.0, 0.5, 10))
ard_FindK(X_train, y_train, 100, 10 ** np.linspace(2.0, 2.5, 10))
ard_FindK(X_train, y_train, 150, 10 ** np.linspace(2.5, 3.0, 10))



ARDRegression
k=50, R-square=0.784213
k=100, R-square=0.983576
k=150, R-square=0.989937
k=150, R-square=0.990735


3) SubMod

In [ ]:
import os
os.chdir("Q3/codetoshare")
import Solver
import submodRegression
from Solver import solvePosteriorPrecisionSparseGreedy

print ("C is Identity:")
def submod_Identity(k):
    square = 0
    lr = linear_model.LinearRegression()
    for i, (train, test) in enumerate(k_fold.split(X_train, y_train)):
        coef = submodRegression.run_submodregression(X_train, y_train, k)
        lr.fit(X_train[train], y_train[train])
        lr.coef_ = coef
        square = square + lr.score(X_train[test], y_train[test])
    print ("k=%d, R-square=%f"% (k, square/5))  

submod_Identity(50)
submod_Identity(100)
submod_Identity(150)

def run_submodregression(Xtrain, ytrain, k, sigma=1):
    r = np.dot(np.transpose(Xtrain), ytrain)/sigma
    C= np.eye(np.shape(Xtrain)[1])*2

    diag_xTx = np.array([0.0] * np.shape(Xtrain)[1])
    for ii in range(len(diag_xTx)):
        diag_xTx[ii] = np.dot(Xtrain[:, ii], Xtrain[:, ii])
#     print "done"

    where1, pmu= solvePosteriorPrecisionSparseGreedy(X = Xtrain, C=C, k = k, r=r, noiseVar=sigma, diag_xTx = diag_xTx, debug=0,opt_dict=None)

    beta = np.zeros(np.shape(Xtrain)[1])
    beta[where1] = pmu
    return beta

def submod_K(k):
    square = 0
    lr = linear_model.LinearRegression()
    for i, (train, test) in enumerate(k_fold.split(X_train, y_train)):
        coef = run_submodregression(X_train, y_train, k)
        lr.fit(X_train[train], y_train[train])
        lr.coef_ = coef
        square = square + lr.score(X_train[test], y_train[test])
    print ("k=%d, R-square=%f"% (k, square/5))  

print ("C is a different Diagonal Matrix:")
submod_K(50)
submod_K(100)
submod_K(150)


C is Identity:
k=50, R-square=0.923545
k=100, R-square=0.998770
k=150, R-square=0.999065
C is a different Diagonal Matrix:
k=50, R-square=0.923547
k=100, R-square=0.998774


# Question 4: Finding Decision Boundary (7 points)
(1) (3 pts) Suppose samples in $R^2$ (the two-dimensional Cartesian space) are being obtained from two classes,
C1 and C2, both of which are normally distributed with means at (1.5, 1) and (1, 1.5) respectively.
The covariance matrix for each class is the same:
$$\Sigma_1 = \Sigma_2 = \begin{bmatrix} 4 & 0 \\ 0 & 4
\end{bmatrix}$$

If the priors of C1 and C2 are 4/7 and 3/7 respectively, what is the ideal (i.e. Bayes Optimal)
decision boundary? (derive the equation for this boundary)

(2) (4 pts) Suppose the cost of misclassifying an input actually belonging to C2 is twice as expensive as
misclassifying an input belonging to C1. Correct classification does not incur any cost. If the
objective is to minimize the expected cost rather than expected misclassification rate, what would
be the best decision boundary? (obtain the equation describing this boundary).

(1)

At the decision boundary:
$$ P(C_1|x) = P(C_2|x) $$
and
$$ P(C_1|x) = P(x|C_1)P(C_1)/p(x) $$
$$ P(C_2|x) = P(x|C_2)P(C_2)/p(x)  $$
so we get:
$$ P(x|C_1)P(C_1) = P(x|C_2)P(C_2)$$
since
$$ P(C_1) = 4/7, P(C_2) = 3/7 $$
$$ P(x|C_1) =  \frac{1}{2\pi |\sum_1|^{1/2} } e ^{-\frac{1}{2}(x - \mu_1)^{T} \sum_1^{-1}(x - \mu_1)} $$
$$ P(x|C_2) =  \frac{1}{2\pi |\sum_2|^{1/2} } e ^{-\frac{1}{2}(x - \mu_2)^{T} \sum_2^{-1}(x - \mu_2)} $$
then
$$ \frac{4}{7} \frac{1}{2\pi |\sum_1|^{1/2} } e ^{-\frac{1}{2}(x - \mu_1)^{T} \sum_1^{-1}(x - \mu_1)} = \frac{3}{7} \frac{1}{2\pi |\sum_2|^{1/2} } e ^{-\frac{1}{2}(x - \mu_2)^{T} \sum_2^{-1}(x - \mu_2)}$$
so the equation is:
$$ 4e^{\frac{(x_1 - 1.5)^2 + (x_2 - 1)^2}{8}} =  3e^{\frac{(x_1 - 1)^2 + (x_2 - 1.5)^2}{8}} $$

(2)

The cost of misclassifying C2 is more expensive, in order to minimize the expected cost:
$$ P(C_1|x) = 2P(C_2|x) $$
implement the same process from (1), we get:
$$ 2e^{\frac{(x_1 - 1.5)^2 + (x_2 - 1)^2}{8}} =  3e^{\frac{(x_1 - 1)^2 + (x_2 - 1.5)^2}{8}} $$